In [1]:
def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

!pip install import_ipynb 
import import_ipynb  
connect_to_drive()
%cd '/content/gdrive/My Drive/Bioinformatics2'

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=c8c0761c15242f5a39083a656b626569882bdb571a9ce54e97bcfc93b5bd8b41
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
Mounted at /content/gdrive
/content/gdrive/My Drive/Bioinformatics2


In [3]:
from pathlib import Path
import os
import cv2
import argparse
import albumentations.augmentations.functional as F
from utils import *
import torch
import cv2

In [4]:
def crop_images(img_list, output_path, txt_cropped_inputs_path, txt_cropped_labels_path):

    Path(out_path).mkdir(parents=True, exist_ok=True)
    for idx, img_path in enumerate(img_list):

        image=cv2.imread(str(img_path))
        seg_path=str(img_path).replace('crop', 'seg_crop')
        seg = cv2.imread(seg_path)
        if seg is None:
          seg_path=seg_path.replace('seg_crop','segmentation_crop')
          seg = cv2.imread(seg_path)
          if seg is None:
            seg_path=seg_path.replace('segmentation_crop','segmentation-crop')
            seg = cv2.imread(seg_path)

        #image = F.pad(image, min_height=IMG_DIM, min_width=IMG_DIM)
        #seg = F.pad(seg, min_height=IMG_DIM, min_width=IMG_DIM)
        

        tile = 0
        for i in range(0, 2000, SLICE_LEN):
            for j in range(0, 2000, SLICE_LEN):
                tile += 1
                path_img=os.path.join(out_path,str(img_path).split('/')[-1].split('.')[0]+f"_{tile:02}.png")
                path_seg=os.path.join(out_path, str(seg_path).split('/')[-1].split('.')[0]+f"_{tile:02}.png")
                print(path_img)
                print(path_seg)
                cv2.imwrite(path_img,
                            image[i:i + INPUT_DIM, j:j + INPUT_DIM])
                cv2.imwrite(path_seg,
                            seg[i:i + INPUT_DIM, j:j + INPUT_DIM])

def crop_squared_tiles_from_image(image, slice_length):
  '''
  It takes a PIL Image and returns an ordered list of cropped squared PIL Images of side slice_length from the original image. 
  Make sure that slice_length is divisible for the image length and width. 
  '''
  squared_tiles=[]
  image_width, image_height, image_depth = image.shape
  for i in range(0, image_width, slice_length):
    for j in range(0, image_height, slice_length):
      squared_tiles.append(image[i:i + slice_length, j:j + slice_length])
  return squared_tiles

def save_crop_squared_tiles_from_path_list(img_path_list, output_folder, txt_file_path, slice_length):
  '''
  It takes a list of images paths, crops every image of the list into squared tiles of sides slice_length, saves them into output 
  folder and saves their name into txt_file_path
  '''
  path_tiles=[]
  for img_path in img_path_list:
    image=cv2.imread(img_path)
    squared_tiles=crop_squared_tiles_from_image(image, slice_length)
    for tile_number,tile in enumerate(squared_tiles):
      path_tile=os.path.join(output_folder,str(img_path).split('/')[-1].split('.')[0]+f"_{tile_number:02}.png")
      path_tile=str(path_tile)
      path_tiles.append(path_tile)
      cv2.imwrite(path_tile, tile)
  save_paths_images(path_tiles, txt_file_path)


In [6]:
test_input_path='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_inputs.txt'
test_target_path='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_targets.txt'
test_input_list=read_list_images_path_from_txt(test_input_path)
test_target_list=read_list_images_path_from_txt(test_target_path)

slice_length=1000
output_folder = '/content/gdrive/MyDrive/Bioinformatics2/validation_dataset_1000/'
txt_file_input_path = '/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_inputs_1000.txt'
txt_file_target_path = '/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_targets_1000.txt'

save_crop_squared_tiles_from_path_list(test_input_list, output_folder, txt_file_input_path, slice_length)
save_crop_squared_tiles_from_path_list(test_target_list, output_folder, txt_file_target_path, slice_length)